In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ============================================================
# REAL-ONLY FINE-TUNED MODEL -> SYNTHETIC TEST EVAL
#   - Model ckpt: best_gaze_finetuned_REAL_ONLY_from_last.pt
#   - Veri: sadece SYNTHETIC test split
#   - Ölçüm: angular_error_deg (normalized gaze vectors)
# ============================================================

import os, json, random, numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T, models
from tqdm import tqdm

# ---------------- Paths ----------------
SYNTH_ROOT = "/content/drive/MyDrive/dataset_split/synthetic_data/left"
CKPT_PATH  = "/content/drive/MyDrive/dataset_split/best_gaze_finetuned_REAL_ONLY_from_last.pt"

# ---------------- Env ----------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

torch.manual_seed(42)
if device == "cuda":
    torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)

# ============================================================
# 1) Euler -> Gaze Vector (TEyeD-style, normalize)
# ============================================================

def euler_xyz_to_matrix(x, y, z):
    cx, cy, cz = np.cos(x), np.cos(y), np.cos(z)
    sx, sy, sz = np.sin(x), np.sin(y), np.sin(z)

    Rx = np.array([[1,0,0],
                   [0,cx,-sx],
                   [0,sx, cx]])

    Ry = np.array([[ cy,0,sy],
                   [  0,1, 0],
                   [-sy,0,cy]])

    Rz = np.array([[cz,-sz,0],
                   [sz, cz,0],
                   [ 0,  0,1]])

    return Rz @ Ry @ Rx

def new_euler_to_gaze_vector(x, y, z):
    """
    Blender XYZ Euler -> forward (-Z) -> TEyeD koordinat sistemi:
      g_blender = R * [0, 0, -1]
      g_teyed   = [ g.x, -g.y, -g.z ]
    Sonrasında normalize.
    """
    R = euler_xyz_to_matrix(x, y, z)
    d_local = np.array([0.0, 0.0, -1.0], dtype=np.float32)
    g = R @ d_local
    g = np.array([g[0], -g[1], -g[2]], dtype=np.float32)
    n = np.linalg.norm(g)
    if n < 1e-8:
        return 0.0, 0.0, -1.0
    g /= n
    return float(g[0]), float(g[1]), float(g[2])

# ============================================================
# 2) Synthetic Dataset (RGBOnlyDataset)
# ============================================================

class RGBOnlyDataset(Dataset):
    """
    Synthetic:
      SYNTH_ROOT/split/render_X/
        frame_001.png .. frame_125.png
        render_metadata.json (rotation_over_time: {frame: {x,y,z}})
    """
    def __init__(self, root_dir, split='test', transform=None):
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform
        self.samples = []  # (frame_path, (gx,gy,gz))

        if not os.path.exists(self.root_dir):
            print(f"⚠️ Synthetic path not found: {self.root_dir}")
            return

        for fld in sorted(os.listdir(self.root_dir)):
            render_path = os.path.join(self.root_dir, fld)
            js = os.path.join(render_path, "render_metadata.json")
            if not os.path.exists(js):
                continue

            try:
                with open(js, "r") as f:
                    meta = json.load(f)
                rot = meta.get("rotation_over_time", {})

                for i in range(1, 126):
                    fp = os.path.join(render_path, f"frame_{i:03d}.png")
                    if not os.path.exists(fp) or str(i) not in rot:
                        continue
                    r = rot[str(i)]
                    if not all(k in r for k in ("x","y","z")):
                        continue

                    ex, ey, ez = float(r["x"]), float(r["y"]), float(r["z"])
                    gx, gy, gz = new_euler_to_gaze_vector(ex, ey, ez)
                    self.samples.append((fp, (gx, gy, gz)))
            except Exception as e:
                print(f"Parse error in {render_path}: {e}")

        print(f"SYNTH-{split}: {len(self.samples)} samples")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        fp, gaze = self.samples[idx]
        img = Image.open(fp).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        tgt = torch.tensor(gaze, dtype=torch.float32)
        return img, tgt

# ============================================================
# 3) Transforms (ImageNet normalize, grayscale->3ch)
# ============================================================

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

synth_tf_eval = T.Compose([
    T.Grayscale(num_output_channels=3),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# ============================================================
# 4) DataLoader (SYNTH TEST)
# ============================================================

synth_test_dataset = RGBOnlyDataset(SYNTH_ROOT, 'test', synth_tf_eval)

BATCH_SIZE = 8
NUM_WORKERS = 4 if device=="cuda" else 2

def make_loader(ds, shuffle):
    return DataLoader(
        ds,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=(device=="cuda"),
        persistent_workers=(device=="cuda" and NUM_WORKERS>0 and len(ds)>0),
        prefetch_factor=4 if device=="cuda" else 2
    )

synth_test_loader = make_loader(synth_test_dataset, shuffle=False)
print(f"SYNTH Test: {len(synth_test_dataset)} samples")

# ============================================================
# 5) Metric: Angular Error (normalized vectors)
# ============================================================

def angular_error_deg(pred, gt):
    pred = F.normalize(pred, dim=1)
    gt   = F.normalize(gt,   dim=1)
    cos_sim = torch.clamp(torch.sum(pred * gt, dim=1), -1.0, 1.0)
    return torch.rad2deg(torch.acos(cos_sim))

# ============================================================
# 6) Model: ResNet50 -> 3D gaze vector, REAL-only ckpt yükle
# ============================================================

gaze_model = models.resnet50(weights=None)  # ckpt tüm ağırlıkları dolduracak
gaze_model.fc = nn.Linear(gaze_model.fc.in_features, 3)
gaze_model = gaze_model.to(device)

ckpt = torch.load(CKPT_PATH, map_location="cpu")
sd = ckpt.get("state_dict", ckpt)
rep = gaze_model.load_state_dict(sd, strict=False)
print("Checkpoint loaded from:", CKPT_PATH)
print("Missing keys:", rep.missing_keys)
print("Unexpected keys:", rep.unexpected_keys)

gaze_model.eval()

# ============================================================
# 7) Test on SYNTHETIC TEST SET
# ============================================================

test_angles_synth = []
with torch.no_grad():
    for xs, ts in tqdm(synth_test_loader, desc="SYNTH Test (REAL-only model)"):
        xs, ts = xs.to(device), ts.to(device)
        out = gaze_model(xs)
        ang = angular_error_deg(out, ts)   # normalized angular error
        test_angles_synth.append(ang.mean().item())

if len(test_angles_synth) > 0:
    mean_ang = float(np.mean(test_angles_synth))
    print(f"✅ REAL-ONLY FINETUNED MODEL on SYNTH TEST Angular Error: {mean_ang:.2f}°")
else:
    print("⚠️ SYNTH TEST dataset boş; hiçbir batch işlenmedi.")


Device: cuda
SYNTH-test: 625 samples
SYNTH Test: 625 samples
Checkpoint loaded from: /content/drive/MyDrive/dataset_split/best_gaze_finetuned_REAL_ONLY_from_last.pt
Missing keys: []
Unexpected keys: []


SYNTH Test (REAL-only model): 100%|██████████| 79/79 [03:24<00:00,  2.59s/it]

✅ REAL-ONLY FINETUNED MODEL on SYNTH TEST Angular Error: 9.57°
